# FashionMNIST in Pytorch

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):  # Create a class that inherits from nn.Module
    def __init__(self):  # Define the layers here
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),

            nn.Linear(512, 10)
        )

    def forward(self, x): # Specify how data will pass through the network
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# Define the loss function and the optimizer to be used
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303300  [    0/60000]
loss: 2.296838  [ 6400/60000]
loss: 2.280193  [12800/60000]
loss: 2.272576  [19200/60000]
loss: 2.274331  [25600/60000]
loss: 2.232123  [32000/60000]
loss: 2.252951  [38400/60000]
loss: 2.215547  [44800/60000]
loss: 2.211100  [51200/60000]
loss: 2.194458  [57600/60000]
Test Error: 
 Accuracy: 28.9%, Avg loss: 2.180781 

Epoch 2
-------------------------------
loss: 2.185707  [    0/60000]
loss: 2.185253  [ 6400/60000]
loss: 2.134240  [12800/60000]
loss: 2.148513  [19200/60000]
loss: 2.120784  [25600/60000]
loss: 2.044983  [32000/60000]
loss: 2.097878  [38400/60000]
loss: 2.013306  [44800/60000]
loss: 2.018078  [51200/60000]
loss: 1.968140  [57600/60000]
Test Error: 
 Accuracy: 54.0%, Avg loss: 1.954492 

Epoch 3
-------------------------------
loss: 1.977011  [    0/60000]
loss: 1.958845  [ 6400/60000]
loss: 1.853026  [12800/60000]
loss: 1.891741  [19200/60000]
loss: 1.803091  [25600/60000]
loss: 1.729599  [32000/600

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Notes on this (non convolutional) model.
- The model predicts the logits, we need to apply Softmax if we want probablities
-  the **nn.Flatten** layer converts each 2D 28x28 image into a contiguous array of 784 pixel values
- The **nn.Linear** layer is a module that applies a linear transformation on the input using its stored weights and biases.
- In this model, we use **nn.ReLU** between our linear layers, but there’s other activations to introduce non-linearity in your model
- **nn.Sequential** is an ordered container of modules. The data is passed through all the modules in the same order as defined.
- The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the **nn.Softmax** module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0349,  0.0309, -0.0213,  ...,  0.0177,  0.0136, -0.0354],
        [-0.0075, -0.0053, -0.0160,  ..., -0.0065, -0.0213, -0.0218]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0134,  0.0039], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0058, -0.0204, -0.0410,  ...,  0.0318,  0.0037, -0.0030],
        [ 0.0168, -0.0428, -0.0299,  ...,  0.0141, -0.0331, -0.0358]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 